# `datascience` Library Demo Notebook

_Notebook created by Chris Pyles_

This notebook is intended to give you some basic information on manipulating rectangular data using the `datascience` library. The `datascience` library is a module for Python developed at UC Berkeley and which is used in the course Data 8: Foundations of Data Science. This notebook covers basic table operations using this library.

<!--

**Table of Contents**
1. [Dependences](#Dependencies)
2. [Loading Data](#Loading-Data)
3. [Moving Between `pandas` and `datascience`](#Moving-Between-pandas-and-datascience)
4. [Rows and Columns](#Rows-and-Columns)
5. [Accessing Vaues](#Accessing-Values)
6. [Missing Values](#Missing-Values)
7. [Descriptive Statistics](#Descriptive-Statistics)
8. [Grouping](#Grouping)
9. [Manipulating Values](#Manipulating-Values)
10. [Exporting Figures](#Exporting-Figures)
11. [Exporting Data](#Exporting-Data)
12. [Conclusion](#Conclusion)

-->

### Dependencies

In the cell below we load the dependencies for this notebook.

In [28]:
from datascience import *
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('fivethirtyeight')
from IPython.display import display

### Loading Data

The method that `datascience` provides for reading in data defaults to reading CSV files. The function, `Table.read_table()`, takes as its argument a relative path to the data file. In the cell below, we load the datasets we will be using for this demo.

In [21]:
trips = Table.read_table('data/trips.csv')
stations = Table.read_table('data/stations.csv')
trips.show(5)
stations.show(5)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/datascience/tables.py:132: FutureWarning: read_table is deprecated, use read_csv instead.
  df = pandas.read_table(filepath_or_buffer, *args, **vargs)


Trip ID,Duration,Start Date,Start Station,Start Terminal,End Date,End Station,End Terminal,Bike #,Subscriber Type,Zip Code
913460,765,8/31/2015 23:26,Harry Bridges Plaza (Ferry Building),50,8/31/2015 23:39,San Francisco Caltrain (Townsend at 4th),70,288,Subscriber,2139
913459,1036,8/31/2015 23:11,San Antonio Shopping Center,31,8/31/2015 23:28,Mountain View City Hall,27,35,Subscriber,95032
913455,307,8/31/2015 23:13,Post at Kearny,47,8/31/2015 23:18,2nd at South Park,64,468,Subscriber,94107
913454,409,8/31/2015 23:10,San Jose City Hall,10,8/31/2015 23:17,San Salvador at 1st,8,68,Subscriber,95113
913453,789,8/31/2015 23:09,Embarcadero at Folsom,51,8/31/2015 23:22,Embarcadero at Sansome,60,487,Customer,9069


station_id,name,lat,long,dockcount,landmark,installation
2,San Jose Diridon Caltrain Station,37.3297,-121.902,27,San Jose,8/6/2013
3,San Jose Civic Center,37.3307,-121.889,15,San Jose,8/5/2013
4,Santa Clara at Almaden,37.334,-121.895,11,San Jose,8/6/2013
5,Adobe on Almaden,37.3314,-121.893,19,San Jose,8/5/2013
6,San Pedro Square,37.3367,-121.894,15,San Jose,8/7/2013


We use the `Table.show()` method above to display the first 5 lines of each table. This method defaults to all rows, so calling `trips.show()` would have displayed all 354,152 rows of that table.

If you have files that use other delimeters, you can pass the `sep` argument of `pd.read_csv()` to `Table.read_table()` and the file will be read in correctly.

In [25]:
Table.read_table("data/trips.tsv", sep="\t").show(5)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/datascience/tables.py:132: FutureWarning: read_table is deprecated, use read_csv instead.
  df = pandas.read_table(filepath_or_buffer, *args, **vargs)


Trip ID,Duration,Start Date,Start Station,Start Terminal,End Date,End Station,End Terminal,Bike #,Subscriber Type,Zip Code
913460,765,8/31/2015 23:26,Harry Bridges Plaza (Ferry Building),50,8/31/2015 23:39,San Francisco Caltrain (Townsend at 4th),70,288,Subscriber,2139
913459,1036,8/31/2015 23:11,San Antonio Shopping Center,31,8/31/2015 23:28,Mountain View City Hall,27,35,Subscriber,95032
913455,307,8/31/2015 23:13,Post at Kearny,47,8/31/2015 23:18,2nd at South Park,64,468,Subscriber,94107
913454,409,8/31/2015 23:10,San Jose City Hall,10,8/31/2015 23:17,San Salvador at 1st,8,68,Subscriber,95113
913453,789,8/31/2015 23:09,Embarcadero at Folsom,51,8/31/2015 23:22,Embarcadero at Sansome,60,487,Customer,9069


If you have data formatted in ways other than delimited files, these need to be loaded into `pandas` first before being transferred to `datascience`. An example call is given below.

```python
# load data into pandas
trips_df = pd.read_json("data/trips.json")

# transfer to datascience
trips_tbl = Table.from_df(trips_df)
```

### Moving Between `pandas` and `datascience`

As noted above, it is possible to transfer your data between `pandas` and `datascience`. The functions to do this are provided in the `datascience` library; `Table.from_df()` takes a DataFrame and returns a Table and `Table.to_df()` turns the Table into a DataFrame.

```python
# pandas to datascience
tbl = Table.from_df(df)

# datascience to pandas
df = tbl.to_df()
```

### Rows and Columns 

To get row and column counts, the `datascience` library provides the `num_rows` and `num_columns` attributes, which are self-explanatory.

In [26]:
trips.num_rows, trips.num_columns

(354152, 11)

To access the labels of the columns, `datascience` has `labels`, which is a tuple containing the column labels in numerical index order.

In [6]:
trips.labels

('Trip ID',
 'Duration',
 'Start Date',
 'Start Station',
 'Start Terminal',
 'End Date',
 'End Station',
 'End Terminal',
 'Bike #',
 'Subscriber Type',
 'Zip Code')

To add columns to a table, you pass a single label and set of values to `.with_column()` or a list of labels and pairs to `.with_columns()` (both shown below). **These functions do not edit the original table, so these modifications can only be saved by assigning them to the name of the table or a new variable name.**

In [51]:
# adding a single column
some_random_numbers = np.random.uniform(0, 10, trips.num_rows)
trips.with_column("Random Numbers", some_random_numbers)

Trip ID,Duration,Start Date,Start Station,Start Terminal,End Date,End Station,End Terminal,Bike #,Subscriber Type,Zip Code,Duration^2,Random Numbers
913460,765,8/31/2015 23:26,Harry Bridges Plaza (Ferry Building),50,8/31/2015 23:39,San Francisco Caltrain (Townsend at 4th),70,288,Subscriber,2139,585225,3.97649
913459,1036,8/31/2015 23:11,San Antonio Shopping Center,31,8/31/2015 23:28,Mountain View City Hall,27,35,Subscriber,95032,1073296,9.66107
913455,307,8/31/2015 23:13,Post at Kearny,47,8/31/2015 23:18,2nd at South Park,64,468,Subscriber,94107,94249,4.42099
913454,409,8/31/2015 23:10,San Jose City Hall,10,8/31/2015 23:17,San Salvador at 1st,8,68,Subscriber,95113,167281,2.93978
913453,789,8/31/2015 23:09,Embarcadero at Folsom,51,8/31/2015 23:22,Embarcadero at Sansome,60,487,Customer,9069,622521,0.366959
913452,293,8/31/2015 23:07,Yerba Buena Center of the Arts (3rd @ Howard),68,8/31/2015 23:12,San Francisco Caltrain (Townsend at 4th),70,538,Subscriber,94118,85849,7.04519
913451,896,8/31/2015 23:07,Embarcadero at Folsom,51,8/31/2015 23:22,Embarcadero at Sansome,60,363,Customer,92562,802816,5.80036
913450,255,8/31/2015 22:16,Embarcadero at Sansome,60,8/31/2015 22:20,Steuart at Market,74,470,Subscriber,94111,65025,9.63289
913449,126,8/31/2015 22:12,Beale at Market,56,8/31/2015 22:15,Temporary Transbay Terminal (Howard at Beale),55,439,Subscriber,94130,15876,5.18192
913448,932,8/31/2015 21:57,Post at Kearny,47,8/31/2015 22:12,South Van Ness at Market,66,472,Subscriber,94702,868624,6.55253


In [52]:
# adding multiple columns
some_more_random_numbers = np.random.normal(0, 10, trips.num_rows)
trips.with_columns(
    "Random Numbers", some_random_numbers,
    "More Random Numbers", some_more_random_numbers
)

Trip ID,Duration,Start Date,Start Station,Start Terminal,End Date,End Station,End Terminal,Bike #,Subscriber Type,Zip Code,Duration^2,Random Numbers,More Random Numbers
913460,765,8/31/2015 23:26,Harry Bridges Plaza (Ferry Building),50,8/31/2015 23:39,San Francisco Caltrain (Townsend at 4th),70,288,Subscriber,2139,585225,3.97649,-5.04509
913459,1036,8/31/2015 23:11,San Antonio Shopping Center,31,8/31/2015 23:28,Mountain View City Hall,27,35,Subscriber,95032,1073296,9.66107,-3.13004
913455,307,8/31/2015 23:13,Post at Kearny,47,8/31/2015 23:18,2nd at South Park,64,468,Subscriber,94107,94249,4.42099,-5.79655
913454,409,8/31/2015 23:10,San Jose City Hall,10,8/31/2015 23:17,San Salvador at 1st,8,68,Subscriber,95113,167281,2.93978,-9.60411
913453,789,8/31/2015 23:09,Embarcadero at Folsom,51,8/31/2015 23:22,Embarcadero at Sansome,60,487,Customer,9069,622521,0.366959,-13.6186
913452,293,8/31/2015 23:07,Yerba Buena Center of the Arts (3rd @ Howard),68,8/31/2015 23:12,San Francisco Caltrain (Townsend at 4th),70,538,Subscriber,94118,85849,7.04519,-4.20664
913451,896,8/31/2015 23:07,Embarcadero at Folsom,51,8/31/2015 23:22,Embarcadero at Sansome,60,363,Customer,92562,802816,5.80036,15.7849
913450,255,8/31/2015 22:16,Embarcadero at Sansome,60,8/31/2015 22:20,Steuart at Market,74,470,Subscriber,94111,65025,9.63289,4.27791
913449,126,8/31/2015 22:12,Beale at Market,56,8/31/2015 22:15,Temporary Transbay Terminal (Howard at Beale),55,439,Subscriber,94130,15876,5.18192,-9.66671
913448,932,8/31/2015 21:57,Post at Kearny,47,8/31/2015 22:12,South Van Ness at Market,66,472,Subscriber,94702,868624,6.55253,-2.43292


Note that in the `.with_columns()` call, the column labels and values alternate; that is, the call should have the form

```python
tbl.with_columns(
    "Label 1", values_1,
    "Label 2", values_2,
    "Label 3", values_3,
    ...
)
```

It is also important that the values argument(s) have the same number of rows as the table they are being added to. A single value entered as this argument will be broadcast to the entire table, but any length besides 1 or the number of rows in the table will throw an error.

It is also possible to change the labels of columns using the `.relabeled()` method.

In [56]:
trips.relabeled("Duration", "Time")

Trip ID,Time,Start Date,Start Station,Start Terminal,End Date,End Station,End Terminal,Bike #,Subscriber Type,Zip Code,Duration^2
913460,765,8/31/2015 23:26,Harry Bridges Plaza (Ferry Building),50,8/31/2015 23:39,San Francisco Caltrain (Townsend at 4th),70,288,Subscriber,2139,585225
913459,1036,8/31/2015 23:11,San Antonio Shopping Center,31,8/31/2015 23:28,Mountain View City Hall,27,35,Subscriber,95032,1073296
913455,307,8/31/2015 23:13,Post at Kearny,47,8/31/2015 23:18,2nd at South Park,64,468,Subscriber,94107,94249
913454,409,8/31/2015 23:10,San Jose City Hall,10,8/31/2015 23:17,San Salvador at 1st,8,68,Subscriber,95113,167281
913453,789,8/31/2015 23:09,Embarcadero at Folsom,51,8/31/2015 23:22,Embarcadero at Sansome,60,487,Customer,9069,622521
913452,293,8/31/2015 23:07,Yerba Buena Center of the Arts (3rd @ Howard),68,8/31/2015 23:12,San Francisco Caltrain (Townsend at 4th),70,538,Subscriber,94118,85849
913451,896,8/31/2015 23:07,Embarcadero at Folsom,51,8/31/2015 23:22,Embarcadero at Sansome,60,363,Customer,92562,802816
913450,255,8/31/2015 22:16,Embarcadero at Sansome,60,8/31/2015 22:20,Steuart at Market,74,470,Subscriber,94111,65025
913449,126,8/31/2015 22:12,Beale at Market,56,8/31/2015 22:15,Temporary Transbay Terminal (Howard at Beale),55,439,Subscriber,94130,15876
913448,932,8/31/2015 21:57,Post at Kearny,47,8/31/2015 22:12,South Van Ness at Market,66,472,Subscriber,94702,868624


### Accessing Values

For all non-continuous variables, it is usually important to understand the possible values of the variable; that is, to know the variable's _unique_ values. While `datascience` does not have a built-in method, it is a simple thing to export a column as an array and pass it to `np.unique`.

In [27]:
np.unique(trips.column('Start Date'))

array(['1/1/2015 0:25', '1/1/2015 0:28', '1/1/2015 0:30', ...,
       '9/9/2014 9:54', '9/9/2014 9:55', '9/9/2014 9:59'], dtype='<U16')

The `datascience` library provides the `.where()` method to filter rows, which uses a column name and a predicate function.

In [32]:
trips.where("Duration", lambda x: x < 100)

Trip ID,Duration,Start Date,Start Station,Start Terminal,End Date,End Station,End Terminal,Bike #,Subscriber Type,Zip Code
913212,73,8/31/2015 18:39,5th at Howard,57,8/31/2015 18:40,5th at Howard,57,456,Subscriber,94107
912621,99,8/31/2015 15:49,Embarcadero at Vallejo,48,8/31/2015 15:51,Davis at Jackson,42,320,Subscriber,94112
912461,86,8/31/2015 12:42,Beale at Market,56,8/31/2015 12:44,Temporary Transbay Terminal (Howard at Beale),55,406,Subscriber,94010
912403,88,8/31/2015 11:39,Beale at Market,56,8/31/2015 11:40,Temporary Transbay Terminal (Howard at Beale),55,361,Subscriber,94114
911322,84,8/30/2015 12:09,Davis at Jackson,42,8/30/2015 12:10,Davis at Jackson,42,520,Customer,19010
911190,86,8/30/2015 4:32,Steuart at Market,74,8/30/2015 4:33,Steuart at Market,74,608,Customer,95035
911156,70,8/29/2015 20:12,South Van Ness at Market,66,8/29/2015 20:14,South Van Ness at Market,66,564,Customer,44
910168,62,8/28/2015 16:32,Embarcadero at Sansome,60,8/28/2015 16:33,Embarcadero at Sansome,60,535,Customer,18977
909789,89,8/28/2015 10:39,Beale at Market,56,8/28/2015 10:40,Temporary Transbay Terminal (Howard at Beale),55,334,Subscriber,94114
909719,73,8/28/2015 9:50,2nd at South Park,64,8/28/2015 9:51,2nd at Townsend,61,375,Subscriber,94107


The library also provides the `are` class to create predicate functions. Each method of this class returns a boolean function that can be called on a value. For example, if we wanted a function that checked whether or not a value is greater than or equal to 1000, we could use the call below:

In [35]:
are.above_or_equal_to(1000)

You can pass these `are` objects to the `.where()` method to use as predicate functions. This is how students in Data 8 are taught to filter rows.

In [36]:
trips.where("Duration", are.below(100))

Trip ID,Duration,Start Date,Start Station,Start Terminal,End Date,End Station,End Terminal,Bike #,Subscriber Type,Zip Code
913212,73,8/31/2015 18:39,5th at Howard,57,8/31/2015 18:40,5th at Howard,57,456,Subscriber,94107
912621,99,8/31/2015 15:49,Embarcadero at Vallejo,48,8/31/2015 15:51,Davis at Jackson,42,320,Subscriber,94112
912461,86,8/31/2015 12:42,Beale at Market,56,8/31/2015 12:44,Temporary Transbay Terminal (Howard at Beale),55,406,Subscriber,94010
912403,88,8/31/2015 11:39,Beale at Market,56,8/31/2015 11:40,Temporary Transbay Terminal (Howard at Beale),55,361,Subscriber,94114
911322,84,8/30/2015 12:09,Davis at Jackson,42,8/30/2015 12:10,Davis at Jackson,42,520,Customer,19010
911190,86,8/30/2015 4:32,Steuart at Market,74,8/30/2015 4:33,Steuart at Market,74,608,Customer,95035
911156,70,8/29/2015 20:12,South Van Ness at Market,66,8/29/2015 20:14,South Van Ness at Market,66,564,Customer,44
910168,62,8/28/2015 16:32,Embarcadero at Sansome,60,8/28/2015 16:33,Embarcadero at Sansome,60,535,Customer,18977
909789,89,8/28/2015 10:39,Beale at Market,56,8/28/2015 10:40,Temporary Transbay Terminal (Howard at Beale),55,334,Subscriber,94114
909719,73,8/28/2015 9:50,2nd at South Park,64,8/28/2015 9:51,2nd at Townsend,61,375,Subscriber,94107


For a full list of predicate functions, see the [`datascience.predicates` documentation](http://data8.org/datascience/predicates.html).

To sort the rows of a table, use the `.sort()` method. It defaults to ascending, so to get values in `descending` order the `descending` argument must be set to `True`.

In [57]:
trips.sort("Duration", descending=True)

Trip ID,Duration,Start Date,Start Station,Start Terminal,End Date,End Station,End Terminal,Bike #,Subscriber Type,Zip Code,Duration^2
568474,17270400,12/6/2014 21:59,South Van Ness at Market,66,6/24/2015 20:18,2nd at Folsom,62,535,Customer,95531,298266716160000
825850,2137000,6/28/2015 21:50,Market at Sansome,77,7/23/2015 15:27,Yerba Buena Center of the Arts (3rd @ Howard),68,466,Customer,97213,4566769000000
750192,1852590,5/2/2015 6:17,San Antonio Shopping Center,31,5/23/2015 16:53,Castro Street and El Camino Real,32,680,Subscriber,94024,3432089708100
841176,1133540,7/10/2015 10:35,University and Emerson,35,7/23/2015 13:27,University and Emerson,35,262,Customer,94306,1284912931600
522337,720454,10/30/2014 8:29,Redwood City Caltrain Station,22,11/7/2014 15:36,Stanford in Redwood City,25,692,Customer,94010,519053966116
774999,688899,5/20/2015 15:27,Palo Alto Caltrain Station,34,5/28/2015 14:49,California Ave Caltrain Station,36,230,Customer,nil,474581832201
635260,655939,2/8/2015 3:05,San Jose Civic Center,3,2/15/2015 17:17,SJSU 4th at San Carlos,12,132,Customer,89451,430255971721
745640,611240,4/29/2015 9:41,University and Emerson,35,5/6/2015 11:28,San Antonio Shopping Center,31,196,Customer,81,373614337600
635263,602338,2/8/2015 3:09,San Jose Civic Center,3,2/15/2015 2:28,San Jose Civic Center,3,9,Customer,89451,362811066244
779645,594550,5/24/2015 13:33,San Pedro Square,6,5/31/2015 10:42,San Pedro Square,6,39,Customer,95118,353489702500


### Missing Values

The `datascience` library does not currently have the functionality to support working with missing values, although it is possible to transfer your data to `pandas` and use that library's tools.

However, it is possible to combine row filtering with NumPy functions (or `pandas` ones) to do some simple filtering. As an example, if we wanted to filter out rows with missing values in a specific column, we could define our own predicate function as below and then use the `.where()` method to filter rows.

In [40]:
not_nan = lambda x: not pd.isna(x)

trips.where('End Terminal', not_nan)

Trip ID,Duration,Start Date,Start Station,Start Terminal,End Date,End Station,End Terminal,Bike #,Subscriber Type,Zip Code
913460,765,8/31/2015 23:26,Harry Bridges Plaza (Ferry Building),50,8/31/2015 23:39,San Francisco Caltrain (Townsend at 4th),70,288,Subscriber,2139
913459,1036,8/31/2015 23:11,San Antonio Shopping Center,31,8/31/2015 23:28,Mountain View City Hall,27,35,Subscriber,95032
913455,307,8/31/2015 23:13,Post at Kearny,47,8/31/2015 23:18,2nd at South Park,64,468,Subscriber,94107
913454,409,8/31/2015 23:10,San Jose City Hall,10,8/31/2015 23:17,San Salvador at 1st,8,68,Subscriber,95113
913453,789,8/31/2015 23:09,Embarcadero at Folsom,51,8/31/2015 23:22,Embarcadero at Sansome,60,487,Customer,9069
913452,293,8/31/2015 23:07,Yerba Buena Center of the Arts (3rd @ Howard),68,8/31/2015 23:12,San Francisco Caltrain (Townsend at 4th),70,538,Subscriber,94118
913451,896,8/31/2015 23:07,Embarcadero at Folsom,51,8/31/2015 23:22,Embarcadero at Sansome,60,363,Customer,92562
913450,255,8/31/2015 22:16,Embarcadero at Sansome,60,8/31/2015 22:20,Steuart at Market,74,470,Subscriber,94111
913449,126,8/31/2015 22:12,Beale at Market,56,8/31/2015 22:15,Temporary Transbay Terminal (Howard at Beale),55,439,Subscriber,94130
913448,932,8/31/2015 21:57,Post at Kearny,47,8/31/2015 22:12,South Van Ness at Market,66,472,Subscriber,94702


If we wanted to filter rows with missing values in _any_ column, we could iterate through the labels in `Table.labels`, using the `.where()` method to filter on each pass:

In [41]:
for label in trips.labels:
    trips = trips.where(label, not_nan)

### Descriptive Statistics

In order to understand the distribution of your numerical data, it can be very useful to look at descriptive statistics of the values. The `datascience` library allows you to compute statistics on each column of your table, but it requires you to specify which operations you want to run and it does not filter out non-numerical columns.

To use the `datascience` library to get descriptive statistics, use the `.stats()` method; this requires you to specify which statistics you want to use to aggregate each column, which you do by passing a list of functions as the `ops` argument.

In [45]:
# datascience
first_quartile = lambda x: np.quantile(x, 0.25)
third_quartile = lambda x: np.quantile(x, 0.75)
trips.stats(ops = [min, max, np.mean, np.std, first_quartile, third_quartile])

statistic,Trip ID,Duration,Start Date,Start Station,Start Terminal,End Date,End Station,End Terminal,Bike #,Subscriber Type,Zip Code,Duration^2
min,432947,60,1/1/2015 0:25,2nd at Folsom,2,1/1/2015 0:30,2nd at Folsom,2,9,Customer,0,3600
max,913460,1.72704e+07,9/9/2014 9:59,Yerba Buena Center of the Arts (3rd @ Howard),84,9/9/2014 9:59,Yerba Buena Center of the Arts (3rd @ Howard),84,878,Subscriber,nil,2.98267e+14
mean,676962,1046.03,,,58.446,,,58.4218,422.958,,,9.02108e+08
std,138874,30016.9,,,16.7388,,,16.8768,159.841,,,5.013e+11
,557010,342,,,50,,,50,327,,,116964
,798026,739,,,70,,,70,546,,,546121


The default behavior of the `.stats()` method is to show the minimum, maximum, median, and sum.

In [46]:
trips.stats()

statistic,Trip ID,Duration,Start Date,Start Station,Start Terminal,End Date,End Station,End Terminal,Bike #,Subscriber Type,Zip Code,Duration^2
min,432947,60,1/1/2015 0:25,2nd at Folsom,2,1/1/2015 0:30,2nd at Folsom,2,9,Customer,0,3600
max,913460,1.72704e+07,9/9/2014 9:59,Yerba Buena Center of the Arts (3rd @ Howard),84,9/9/2014 9:59,Yerba Buena Center of the Arts (3rd @ Howard),84,878,Subscriber,nil,2.98267e+14
median,679452,511,,,62,,,63,437,,,261121
sum,2.39748e+11,3.70455e+08,,,2.06988e+07,,,2.06902e+07,1.49791e+08,,,3.19483e+14


### Grouping

In the `datascience` library, you can group by a column with the `.group()` method; this defaults to counts, but you can pass an optional second argument with an aggregator function.

In [14]:
trips.group('Start Station')

Start Station,count
2nd at Folsom,7999
2nd at South Park,9469
2nd at Townsend,14026
5th at Howard,7708
Adobe on Almaden,562
Arena Green / SAP Center,647
Beale at Market,8359
Broadway St at Battery St,7676
California Ave Caltrain Station,400
Castro Street and El Camino Real,1230


When you pass an aggregator function, each column is aggregated by that function in the specified groups. This means that the new table will have the same number of columns as the original, unlike the call _without_ an aggregator function. As an example of an aggregator function, we could pass `np.median()`.

In [47]:
trips.group("Start Station", np.median)

Start Station,Trip ID median,Duration median,Start Date median,Start Terminal median,End Date median,End Station median,End Terminal median,Bike # median,Subscriber Type median,Zip Code median,Duration^2 median
2nd at Folsom,669563,350,,62,,,65,448,,,122500
2nd at South Park,657212,446,,64,,,68,449,,,198916
2nd at Townsend,677760,488,,61,,,61,448,,,238144
5th at Howard,666768,437,,57,,,69,449,,,190969
Adobe on Almaden,689409,329.5,,5,,,2,184.5,,,108570
Arena Green / SAP Center,652938,375,,14,,,6,178,,,140625
Beale at Market,670975,450,,56,,,64,448,,,202500
Broadway St at Battery St,670514,470,,82,,,69,446,,,220900
California Ave Caltrain Station,704463,1085,,36,,,35,218,,,1.17723e+06
Castro Street and El Camino Real,689918,308,,32,,,28,188,,,94864


To create a pivot table, use the `.pivot()` method. The first argument indicates the column labels, the second the rows, and the third the values that go into each entry. If there are more than one value to go into the cells, it is also possible to pass an aggregator function. The cell below shows a table where each column is a starting station, each row is an ending station, and each value is the mean of the durations for that starting and ending station pair.

In [58]:
trips.pivot("Start Station", "End Station", "Duration", np.mean)

End Station,2nd at Folsom,2nd at South Park,2nd at Townsend,5th at Howard,Adobe on Almaden,Arena Green / SAP Center,Beale at Market,Broadway St at Battery St,California Ave Caltrain Station,Castro Street and El Camino Real,Civic Center BART (7th at Market),Clay at Battery,Commercial at Montgomery,Cowper at University,Davis at Jackson,Embarcadero at Bryant,Embarcadero at Folsom,Embarcadero at Sansome,Embarcadero at Vallejo,Evelyn Park and Ride,Franklin at Maple,Golden Gate at Polk,Grant Avenue at Columbus Avenue,Harry Bridges Plaza (Ferry Building),Howard at 2nd,Japantown,MLK Library,Market at 10th,Market at 4th,Market at Sansome,Mechanics Plaza (Market at Battery),Mezes Park,Mountain View Caltrain Station,Mountain View City Hall,Palo Alto Caltrain Station,Park at Olive,Paseo de San Antonio,Post at Kearny,Powell Street BART,Powell at Post (Union Square),Redwood City Caltrain Station,Redwood City Medical Center,Redwood City Public Library,Rengstorff Avenue / California Street,Ryland Park,SJSU - San Salvador at 9th,SJSU 4th at San Carlos,San Antonio Caltrain Station,San Antonio Shopping Center,San Francisco Caltrain (Townsend at 4th),San Francisco Caltrain 2 (330 Townsend),San Francisco City Hall,San Jose City Hall,San Jose Civic Center,San Jose Diridon Caltrain Station,San Mateo County Center,San Pedro Square,San Salvador at 1st,Santa Clara County Civic Center,Santa Clara at Almaden,South Van Ness at Market,Spear at Folsom,St James Park,Stanford in Redwood City,Steuart at Market,Temporary Transbay Terminal (Howard at Beale),Townsend at 7th,University and Emerson,Washington at Kearny,Yerba Buena Center of the Arts (3rd @ Howard)
2nd at Folsom,3126.6,393.552,334.699,445.546,0,0,595.537,839.636,0,0,790.064,516.897,588.491,0,1761.4,433.744,533.061,2260.81,9999.29,0,0,825,1612.61,2456.6,568.745,0,0,747.494,460.217,317.875,378.576,0,0,0,0,0,0,1635.44,567.789,518.011,0,0,0,0,0,0,0,0,0,473.157,484.382,1094.05,0,0,0,0,0,0,0,0,432645,365.579,0,0,1061.83,455.913,693.786,0,617,482.438
2nd at South Park,235.729,1671.71,316.23,542.295,0,0,529.043,715.247,0,0,926.43,680.189,670.073,0,640.081,567.283,404.27,1134.38,648.948,0,0,754.362,993.987,873.881,240.62,0,0,841.8,575.401,332.44,423.605,0,0,0,0,0,0,347.313,804.831,626.8,0,0,0,0,0,0,0,0,0,302.737,448.294,883.857,0,0,0,0,0,0,0,0,831.879,380.366,0,0,564.244,367.077,485.868,0,722.063,440.769
2nd at Townsend,434.341,583.755,2636.55,655.196,0,0,689.255,780.067,0,0,846.888,846.514,850.543,0,775.514,483.728,597.793,1239.13,1870.3,0,0,1070.59,981.578,645.451,520.583,0,0,4071.93,1134.56,523.379,873.341,0,0,0,0,0,0,1432.18,2074.36,776.333,0,0,0,0,0,0,0,0,0,424.364,436.993,1882.94,0,0,0,0,0,0,0,0,1148.03,422.858,0,0,551.893,543.003,569.572,0,1228.74,595.311
5th at Howard,501.259,999.304,1660.5,6787.49,0,0,973.79,883.681,0,0,455.276,692.623,718.435,0,1161.85,747.672,613.49,1262.89,2632.8,0,0,520.207,834.692,767.9,391.229,0,0,702.511,662.145,467.976,671.872,0,0,0,0,0,0,525.989,829.788,422.584,0,0,0,0,0,0,0,0,0,456.753,395.332,703.86,0,0,0,0,0,0,0,0,621.237,584.98,0,0,800.065,411.037,569.475,0,1031.8,808.459
Adobe on Almaden,0,0,0,0,3778.86,3475.33,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,942.176,2026.25,0,0,0,0,0,0,0,0,0,825,0,0,0,0,0,0,0,2441.27,858.571,534.438,0,0,0,0,0,546.316,3098.36,838.078,0,360.4,2353.5,1164.4,681.667,0,0,516.857,0,0,0,0,0,0,0
Arena Green / SAP Center,0,0,0,0,1506.11,4824.14,0,0,0,0,0,0,0,0,0,0,0,0,0,27409,0,0,0,0,0,1037.53,1692.33,0,0,0,0,0,0,0,0,0,1013.91,0,0,0,0,0,0,0,836.731,902.333,1119.12,0,0,0,0,0,536.667,845.476,1109.33,0,373.393,1127.92,1307.16,283.294,0,0,645,0,0,0,0,0,0,0
Beale at Market,360.701,603.873,788.027,916.781,0,0,6228.8,534.03,0,0,627.423,637.938,303.168,0,232.777,441.483,379.897,610.12,450.494,0,0,697.074,480.713,2682.78,329.5,0,0,731.175,409.882,564.178,1748.41,0,0,0,0,0,0,1996.71,502.335,547.462,0,0,0,0,0,0,0,0,0,788.271,810.31,809.92,0,0,0,0,0,0,0,0,923.344,377.338,0,0,1351.05,466.977,953.429,0,753.415,552.587
Broadway St at Battery St,697.913,865.663,744.826,

### Joining Tables

The `datascience` library allows you to join tables using its `.join()` method. This method performs an _inner_ join, which means that the rows are only those whose values in the join column(s) appear in _both_ tables. 

The call below joins the `trips` table with the second through fourth columns of the `stations` table, left on `"Start Station"` and right on `"name"`. This means that the result table will have two new columns, `"lat"` and `"long"`, indicating the latitude and longitude of the starting station.

In [62]:
trips.join("Start Station", stations.select(1, 2, 3), "name")

Start Station,Trip ID,Duration,Start Date,Start Terminal,End Date,End Station,End Terminal,Bike #,Subscriber Type,Zip Code,Duration^2,lat,long
2nd at Folsom,913183,226,8/31/2015 18:30,62,8/31/2015 18:34,Market at Sansome,77,621,Subscriber,94612,51076,37.7853,-122.396
2nd at Folsom,913145,461,8/31/2015 18:16,62,8/31/2015 18:24,San Francisco Caltrain 2 (330 Townsend),69,606,Subscriber,95125,212521,37.7853,-122.396
2nd at Folsom,913135,399,8/31/2015 18:13,62,8/31/2015 18:19,Clay at Battery,41,421,Subscriber,94920,159201,37.7853,-122.396
2nd at Folsom,913107,283,8/31/2015 18:03,62,8/31/2015 18:07,Market at Sansome,77,375,Subscriber,94549,80089,37.7853,-122.396
2nd at Folsom,913100,263,8/31/2015 18:01,62,8/31/2015 18:06,2nd at Townsend,61,621,Subscriber,94107,69169,37.7853,-122.396
2nd at Folsom,913092,278,8/31/2015 18:00,62,8/31/2015 18:04,Temporary Transbay Terminal (Howard at Beale),55,518,Subscriber,94710,77284,37.7853,-122.396
2nd at Folsom,913056,437,8/31/2015 17:51,62,8/31/2015 17:59,Steuart at Market,74,193,Subscriber,94114,190969,37.7853,-122.396
2nd at Folsom,913046,309,8/31/2015 17:49,62,8/31/2015 17:55,San Francisco Caltrain (Townsend at 4th),70,432,Subscriber,94305,95481,37.7853,-122.396
2nd at Folsom,913041,330,8/31/2015 17:48,62,8/31/2015 17:53,Harry Bridges Plaza (Ferry Building),50,601,Subscriber,94960,108900,37.7853,-122.396
2nd at Folsom,913033,379,8/31/2015 17:47,62,8/31/2015 17:53,Steuart at Market,74,191,Subscriber,94563,143641,37.7853,-122.396


To perform other types of joins, the tables would need to be passed to `pandas`.

### Manipulating Values

The most common way to manipulate a data set is to apply a predefined function on each element of a column. To accomplish this in `datascience`, we utilize the `.apply()` method, which takes as its arguments first a function to apply and then the column index or label.

In [42]:
square = lambda x: x**2

sqaured_durations = trips.apply(square, "Duration")
trips = trips.with_column("Duration^2", sqaured_durations)
trips.show(5)

Trip ID,Duration,Start Date,Start Station,Start Terminal,End Date,End Station,End Terminal,Bike #,Subscriber Type,Zip Code,Duration^2
913460,765,8/31/2015 23:26,Harry Bridges Plaza (Ferry Building),50,8/31/2015 23:39,San Francisco Caltrain (Townsend at 4th),70,288,Subscriber,2139,585225
913459,1036,8/31/2015 23:11,San Antonio Shopping Center,31,8/31/2015 23:28,Mountain View City Hall,27,35,Subscriber,95032,1073296
913455,307,8/31/2015 23:13,Post at Kearny,47,8/31/2015 23:18,2nd at South Park,64,468,Subscriber,94107,94249
913454,409,8/31/2015 23:10,San Jose City Hall,10,8/31/2015 23:17,San Salvador at 1st,8,68,Subscriber,95113,167281
913453,789,8/31/2015 23:09,Embarcadero at Folsom,51,8/31/2015 23:22,Embarcadero at Sansome,60,487,Customer,9069,622521


### Exporting Data

If you make some modifications to the data set or do some data cleaning, you may want to export your data from Python to make it easier to pick up later or to reproduce. For this reason, there is a `datascience` function that allow you to export a Table object to a text file, which you can then load back into Python later. To export as a CSV file, you pass the file name (or file location, if it's going to another folder) to the `.to_csv()` method.

In [49]:
trips.to_csv('export/trips-export.csv')

If you want to save as another file format (e.g. TSV, JSON), you will have to export through `pandas` by setting the `sep` argument of the `.to_csv()` method or using a different export function (e.g. `pd.to_json()`). This is easily accomplished if you have a Table by transferring that table to `pandas` first.

In [50]:
# transfer to pandas, from above
trips_df = trips.to_df()

# export as tsv
trips_df.to_csv('data/trips.tsv', index=False, sep='\t')

### Conclusion

This notebook should have given you a good introduction to the `Table` class of `datascience`. This demo is not an exhaustive one, and there are _many_ other functionalities of the class that were not covered. To see these and the other functions in the library (including plotting and mapping functionality), see the [`datascience` documentation](http://data8.org/datascience).